### Введение в структуры и классы

Повторение

Что произойдёт ?

```C++
#include <iostream>

class A
{
    A(int value)
        : x(value)
        , y(x * x)
    {
    }

    int x; // value
    int y; // value ^ 2
};

int main()
{
    
    A a(10);
    std::cout<< a.x; // -> что выведет?

    return 0;
}
```

<details>
<summary>Ответ</summary>
<p>
compile error, member is private
</p>
</details>

Что произойдёт ?

```C++
#include <iostream>

class A
{
    A(int value)
        : y(value)
        , x(y * y)
    {
    }

public:
    int x; // value ^ 2
    int y; // value
};

int main()
{
    
    A a(10);
    std::cout<< a.x; // -> что выведет?

    return 0;
}
```

<details>
<summary>Ответ</summary>
<p>
compile error, member init seq
</p>
</details>

Правила автогенерации:

```C++
struct S
{
    char m0[3];
    short int m1;
    long long int m2;
    char m3;
    double m4;
};
```

разместите правильно поля в структуре для минимального размера структуры



<details>
<summary>Ответ</summary>
<p>
```C++
struct S
{
    char m0[3];
    char m2;
    long long int m1;
    double m3;
    int m4
};
```
</p>
</details>

# Семантика указателя


##### Семантика указателя

```c++
class MyObject
{
public:
    void foo() {}
};

class MyObjectPtr
{
    MyObject* ptr_;
public:
    MyObjectPtr()
        : ptr_(new MyObject())
    {
    }

    ~MyObjectPtr()
    {
        delete ptr_;
    }

    MyObject& operator*()
    {
        return *ptr_;
    }

    const MyObject& operator*() const
    {
        return *ptr_;
    }

    MyObject* operator->()
    {
        return ptr_;
    }

    const MyObject* operator->() const
    {
        return ptr_;
    }
};

MyObjectPtr p;
p->foo();
(*p).foo();
```


##### Функтор

Позволяет работать с объектом как с функцией.

```c++
class Less
{
public:
    bool operator()(
        const BigInt& left, const BigInt& right) const
    {
        return left < right;
    }
};

Less less;
if (less(3, 5))
    ...
```


##### Другие операторы

- new
- delete
- ,


### Сокрытие

```c++
struct A
{
    void foo() {} // 1
};

struct B
    : public A
{
    void foo() {} // 2
};

A a;
a.foo(); // Будет вызвана 1

B b;
b.foo(); // Будет вызвана 2

A* c = new B();
c->foo(); // Какая будет вызвана функция?
```


<details>
<summary>Ответ</summary>
Будет вызвана 1
</details>




### Виртуальные функции

```c++
struct A
{
    virtual void foo() const {} // 1
};

struct B
    : public A
{
    void foo() const override {} // 2
};

A a;
a.foo(); // Будет вызвана 1

B b;
b.foo(); // Будет вызвана 2

A* c = new B();
c->foo(); // Будет вызвана 2

const A& d = B();
d.foo(); // Будет вызвана 2
```

В первых двух случаях используется раннее (статическое) связывание, еще на этапе компиляции компилятор знает какой метод вызвать.

В третьем случае используется позднее (динамическое) связывание, компилятор на этапе компиляции не знает какой метод вызвать, выбор нужного метода будет сделан во время выполнения.

#### Виртуальные функции в С

```C++
int main()
{
    Device* devices[] = {
        Console_new(10),
        Socket_new("10.0.0.1") };

    Device* dev1 = devices[0];
    Device_write(dev1, "A");

    Device* dev2 = devices[1];
    Device_write(dev2, "B");

    return 0;
}
...
```


```C
#include <stdio.h>
#include <stdlib.h>

struct Device;

struct DeviceVirtualFunctionTable
{
    void (*write)(Device* self, const char* message);
};

struct Device
{
    DeviceVirtualFunctionTable vft_;
};

void Device_write(Device* self, const char* message)
{
    self->vft_.write(self, message);
}

struct Console
{
    DeviceVirtualFunctionTable vft_;
    int id_;
};

void Console_write(Device* self, const char* message)
{
    Console* console = (Console*) self;
    printf("Console %d: %s\n", console->id_, message);
}

Device* Console_new(int id)
{
    Console* instance = (Console*) malloc(sizeof(Console));
    instance->vft_.write = Console_write;
    instance->id_ = id;
    return (Device*) instance;
}

struct Socket
{
    DeviceVirtualFunctionTable vft_;
    const char* address_;
};

void Socket_write(Device* self, const char* message)
{
    Socket* socket = (Socket*) self;
    printf("Send %s to %s\n", message, socket->address_);
}

Device* Socket_new(const char* address)
{
    Socket* instance = (Socket*) malloc(sizeof(Socket));
    instance->vft_.write = Socket_write;
    instance->address_ = address;
    return (Device*) instance;
}

int main()
{
    Device* devices[] = {
        Console_new(10),
        Socket_new("10.0.0.1") };

    Device* dev1 = devices[0];
    Device_write(dev1, "A");

    Device* dev2 = devices[1];
    Device_write(dev2, "B");

    return 0;
}
```

```
Console 10: A
Send B to 10.0.0.1
```


___
### C++ code

```c++
#include <stdio.h>

struct Device
{
    virtual void write(const char* message) {}
};

class Console : public Device
{
    int id_;
public:
    Console(int id)
        : id_(id)
    {
    }

    void write(const char* message) override
    {
        printf("Console %d: %s\n", id_, message);
    }
};

class Socket : public Device
{
    const char* address_;
public:
    Socket(const char* address)
        : address_(address)
    {
    }

    void write(const char* message) override
    {
        printf("Send %s to %s\n", message, address_);
    }
};

int main()
{
    Device* devices[] = {
        new Console(10),
        new Socket("10.0.0.1") };

    Device* dev1 = devices[0];
    dev1->write("A");

    Device* dev2 = devices[1];
    dev2->write("B");

    return 0;
}
```

```
Console 10: A
Send B to 10.0.0.1
```


##### Таблица виртуальных функций

Если в классе или в каком-либо его базовом классе есть виртуальная функция, то каждый объект хранит указатель на таблицу виртуальных функций.

Таблица представляет собой массив из указателей на функции.

```c++
struct A
{
    void foo() {}
    int x;
};

struct B
{
    virtual void foo() {}
    int x;
};

std::cout << sizeof(A) << '\n';
std::cout << sizeof(B) << '\n';
```

```
4
16
```



##### Виртуальный деструктор

```c++
struct A
{
    ~A()
    {
        std::cout << "A";
    }
};

struct B
    : public A
{
    ~B()
    {
        std::cout << "B";
        delete object_;
    }

    SomeObject* object_;
};

A* a = new B();
delete a;
```

stdout:
```
A
```

> Произошла утечка, так как не был вызван деструктор, в котором мы освобождали ресурс.

```c++
struct A
{
    virtual ~A()
    {
    }
};
```

> Используете наследование? Сделайте деструктор виртуальным.

##### Чисто виртуальные функции (pure virtual)

```c++
class Writer
{
public:
    virtual void ~Writer() {}

    virtual void write(const char* message) = 0;
};

class ConsoleWriter
    : public Writer
{
public:
    void write(const char* message) override
    {
        std::cout << message;
    }
}
```



##### Абстрактные классы

Классы имеющие хоть одну чисто виртуальную функцию - абстрактные. При попытке создать их компилятор выдаст ошибку. Если в производном классе не сделать реализацию чисто виртуальной функции, то он тоже становится абстрактным.

> Абстрактные классы в С++ - продвинутые интерфейсные классы в других языках.


##### Виртуальные функции и параметры по умолчанию

```c++
struct A
{
    virtual void foo(int i = 10)
    {
        std::cout << i; // 1
    }
};

struct B
    : public A
{
    virtual void foo(int i = 20)
    {
        std::cout << i; // 2
    }
};

A* a = new B();
a->foo(); // Будет вызвана 2, вывод 10

B* b = new B();
b->foo(); // Будет вызвана 2, вывод 20

A* a = new A();
a->foo(); // Будет вызвана 1, вывод 10
```

> Лучше избегать параметров по умолчанию для виртуальных функций


### Модификаторы доступа при наследовании

```c++
class A
{
public:
    int x_;
protected:
    int y_;
private:
    int z_;
};
```

Псевдокод! Поля базового класса после наследования имеют такие модификаторы:

```c++
class B : public A
{
public:
    int x_;
protected:
    int y_;
};

A* a = new B(); // Ok
```
```c++
class B : protected A
{
protected:
    int x_;
    int y_;
};

A* a = new B(); // Ошибка
```
```c++
class  B : private A
{
private:
    int x_;
    int y_;
};

A* a = new B(); // Ошибка
```


##### public - классическое ООП наследование

```c++
class Device
{
};

class NetworkAdapter
    : public Device
{
};

class DeviceManager
{
    void addDevice(Device* dev)
    {
    }
}

devManager.addDevice(new NetworkAdapter());
```

##### private - наследование реализации

```c++
class NetworkAdapter
    : public Device
    , private Loggable
{
};

Loggable* l = new NetworkAdapter(); // Ошибка
```

#### final

```c++
struct A final
{
};

struct B : public A // Ошибка
```

### Множественное наследование

```c++
struct A
{
    virtual ~A() {}
    double x;
    double y;
};

struct B : public A { };

struct C : public A { };

struct D
    : public B
    , public C
{
};
```

```
+------+ +------+
|  A   | |  A   |
| x, y | | x, y |
| vtab | | vtab |
+------+ +------+
    ^       ^
    |       |
+------+ +------+
|  B   | |  C   |
+------+ +------+
     ^    ^
     |    |
    +------+
    |  D   |
    +------+
```

```c++
// 2 * 8(double) + 1 * 8(vtable)
sizeof(A) == 24
sizeof(B) == 24
sizeof(C) == 24
// (2 * 8(double) + 1 * 8(vtable)) + (2 * 8(double) + 1 * 8(vtable))
sizeof(D) == 48
```

```
[A][B][D]
         [A][C]
```

```c++
struct A
{
    A(double x)
        : x(x)
        , y(0)
    {
    }
    virtual ~A() {}
    double x;
    double y;
};

struct B : public A
{
    B(double x)
        : A(x)
    {
        y = x * 2;
    }
};

struct C : public A
{
    C(double x)
        : A(x)
    {
        y = x * 2;
    }
};

struct D
    : public B
    , public C
{
    D()
        : B(2)
        , C(3)
    {
        B::y = 1;
        C::y = 2;
    }
};
```

#### Ромбовидное наследование

```c++
struct A
{
    virtual ~A() {}
    double x;
    double y;
};

struct B : virtual public A { };

struct C : virtual public A { };

struct D
    : public B
    , public C
{
};
```

```
    +------+
    |  A   |
    | x, y |
    | vtab |
    +------+
     ^    ^
     |    |
+------+ +------+
|  B   | |  C   |
| vtab | | vtab |
+------+ +------+
     ^    ^
     |    |
    +------+
    |  D   |
    +------+
```

```c++
// 2 * 8(double) + 1 * 8(vtable)
sizeof(A) == 24
// 2 * 8(double) + 2 * 8(vtable)
sizeof(B) == 32
sizeof(C) == 32
// 2 * 8(double) + 3 * 8(vtable)
sizeof(D) == 40
```

```
[B][D]
      [C]
         [A]
```



### Вложенные классы

```c++
class Vector
{
public:
    class Iterator
    {
    };

private:
    char* data_;
};

Vector::Iterator it = ...
```

> Имеют доступ к закрытой части внешнего класса


### Практическая часть

Нужно написать класс-матрицу, тип элементов int. В конструкторе задается количество рядов и строк. Поддерживаются оперции: получить количество строк(rows)/столбцов(columns), получить конкретный элемент, умножить на число(*=), сравнение на равенство/неравенство. В случае ошибки выхода за границы бросать исключение:

```c++
throw std::out_of_range("")
```

Пример:

```c++
const size_t rows = 5;
const size_t cols = 3;

Matrix m(rows, cols);

assert(m.getRows() == 5);
assert(m.getColumns() == 3);

m[1][2] = 5; // строка 1, колонка 2
double x = m[4][1];

m *= 3; // умножение на число

Matrix m1(rows, cols);

if (m1 == m)
{
}
```

##### Подсказка

Чтобы реализовать семантику [][] понадобится прокси-класс. Оператор матрицы возращает другой класс, в котором тоже используется оператор [] и уже этот класс возвращает значение.